In [1]:
import os
import pickle
import pandas as pd

In [51]:
cwd = os.getcwd()

In [52]:
top_10_industry = [i for i in os.listdir() if i.endswith('.INDX')]

In [69]:
all_factor_list = []
for ids in top_10_industry:
    for p in [_ for _ in os.listdir(cwd+'\\'+ids) if _.endswith('.pkl')]: 
        with open(f'{ids}/{p}','rb') as pf:
            result = pickle.load(pf)
        config,bt_result = result[0],result[1]
        if bt_result is None:
            continue
        summary = bt_result['sys_analyser']['summary']
        factor = config['extra']['context_vars']['stock_selection_args'].factor
        is_asc = config['extra']['context_vars']['stock_selection_args'].ascending
        factor_dict = {}
        factor_dict['industry'] = ids
        factor_dict['factor'] = p[:-4]
        factor_dict['ascending'] = is_asc
        factor_dict.update({k:v for k,v in summary.items() if k in ['annualized_returns','sharpe','max_drawdown']})
        all_factor_list.append(factor_dict)
all_factor_df = pd.DataFrame(all_factor_list)

In [70]:
def sort_annual(df):
    df = df.sort_values('annualized_returns',ascending=False)
    del df['industry']
    return df[:10]
sorted_df = all_factor_df.groupby('industry').apply(sort_annual)

In [71]:
sorted_df = sorted_df.reset_index().set_index(['industry','factor'])

In [72]:
sorted_df.drop(columns='level_1',inplace=True)

In [73]:
# sorted_df.to_excel(r'best-factor-industry.xlsx')

----

In [74]:
sorted_df = sorted_df.reset_index().set_index('industry')

In [77]:
sorted_df = sorted_df.iloc[::10,0]

In [78]:
sorted_df

industry
801030.INDX                    interest_bearing_debt_ttm
801080.INDX           operating_revenue_growth_ratio_ttm
801140.INDX                              inc_revenue_ttm
801150.INDX        adjusted_return_on_equity_diluted_ttm
801160.INDX                    interest_bearing_debt_ttm
801730.INDX           operating_revenue_growth_ratio_ttm
801750.INDX                 earned_reserve_per_share_ttm
801760.INDX                              ocf_to_debt_ttm
801880.INDX               current_debt_to_total_debt_ttm
801890.INDX    non_interest_bearing_non_current_debt_ttm
Name: factor, dtype: object

In [93]:
for ids,fac in sorted_df.items():
    if 'per_share' in fac:
        continue
    fac +='.pkl'
    with open(f'{ids}/{fac}','rb') as pf:
        result = pickle.load(pf)
        config,bt_result = result[0],result[1]
        trades = bt_result['sys_analyser']['trades']
    trades.to_csv(f'{ids}/{ids}_trades.csv')

In [3]:
ids = '801890.INDX'
fac = 'capital_to_equity_ratio_ttm'

with open(f'{ids}/{fac}.pkl','rb') as pf:
    result = pickle.load(pf)
    config,bt_result = result[0],result[1]
    trades = bt_result['sys_analyser']['trades']
    trades.to_csv(f'{ids}/{fac}_trades.csv')